<a href="https://colab.research.google.com/github/mehran1373/analysis/blob/main/meh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello')